
<div style='text-align:center;color:red;font-size:36px;margin:10px'> CORONA VIRUS ALERT !!!</div>
<div style='margin-top:20px;text-align:center;font-size:16px'>A deadly virus spreading from Human to Human</div>
    

<span style='font-size:20px;color:#FFA500'>--</span><span style='font-size:20px'>Description</span><span style='font-size:20px;color:#FFA500'>--</span><br><br>
<span style='background:yellow'>2019 Novel Coronavirus (2019-nCoV) is a virus (more specifically, a coronavirus) identified as the cause of an outbreak of respiratory illness first detected in Wuhan, China.</span> Early on, many of the patients in the outbreak in Wuhan, China reportedly had some link to a large seafood and animal market, suggesting animal-to-person spread. However, a growing number of patients reportedly have not had exposure to animal markets, indicating person-to-person spread is occurring. At this time, it’s unclear how easily or sustainably this virus is spreading between people - CDC

This dataset has daily level information on the number of affected cases, deaths and recovery from 2019 novel coronavirus. Please note that this is a time series data and so the number of cases on any given day is the cumulative number.

The data is available from 22 Jan, 2020.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec

In [ ]:
dataset = pd.read_csv("/kaggle/input/novel-corona-virus-2019-dataset/2019_nCoV_data.csv")

In [ ]:
dataset.head()

In [ ]:
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset['Last Update'] = pd.to_datetime(dataset['Last Update'])

In [ ]:
dataset['Day'] = dataset['Date'].dt.day
dataset['Month'] = dataset['Date'].dt.month
dataset['Week'] = dataset['Date'].dt.week
dataset['WeekDay'] = dataset['Date'].dt.weekday

In [ ]:
# Layout Customization
displayed_cols = ['Confirmed','Deaths','Recovered']
def multi_plot():
    fig = plt.figure(constrained_layout=True, figsize=(15,8))
    grid = gridspec.GridSpec(ncols=4, nrows=2, figure=fig)

    ax1 = fig.add_subplot(grid[0, :2])
    ax1.set_title('Daily Reports')
    dataset.groupby(['Date']).sum()[displayed_cols].plot(ax=ax1)

    ax2 = fig.add_subplot(grid[1, :2])
    ax2.set_title('Monthly Reports')
    dataset.groupby(['Month']).sum()[displayed_cols].plot(kind='bar',ax=ax2)
    ax2.set_xticklabels(range(1,3))

    ax3= fig.add_subplot(grid[0, 2:])
    ax3.set_title('Weekly Reports')
    weekdays = dataset.groupby('Week').nth(-1)['Date']
    dataset[dataset['Date'].isin(weekdays)].groupby('Date')[displayed_cols].sum().plot(kind='bar',ax=ax3)
    ax3.set_xticklabels(range(1,len(weekdays)+1))

    ax4 = fig.add_subplot(grid[1, 2:])
    ax4.set_title('WeekDays Reports')
    dataset.groupby(['WeekDay']).sum()[displayed_cols].plot(ax=ax4)
    plt.show()
multi_plot()

Daily Report shows that case count is <span style='color:red'> increasing day by day </span> which means that the calculation for <span style='color:green'>today case count = today case count + previous days case count included</span>. So we will take the <span style='color:red'><b>last date</b></span> as the updated case count.

In [ ]:
recent_date = dataset['Date'].iloc[-1]
last_updated = dataset[dataset['Date'].dt.date == recent_date]

In [ ]:
# Reports given for the total number of days
dataset['Date'].max() - dataset['Date'].min()

In [ ]:
pd.DataFrame(dataset['Country'].value_counts()).style.set_table_styles(
[{'selector': 'tr:nth-of-type(-n+5)',
  'props': [('background', '#FFA500')]}, 
 {'selector': 'th',
  'props': [('background', '#606060'), 
            ('color', 'white'),
            ('font-family', 'verdana')]}
]
)

<span style='color:blue'> Observation : </span>

Mainland China and China has two separate entries. We'll combine the both

<span style='background:yellow;color:black'> Description of this given dataset and News Confirms that CoronaVirus was spreaded from China</span> and from the above table after China. US,Australia and Canada has higher entries and geographically these countries are too far from China . It's weired We'll analyse it further

We'll look into data of What the <span style='background:yellow;color:black'>Country name <b>Others</b></span> has

<span style='font-size:20px;color:#FFA500'>--[](http://)</span><span style='font-size:20px'>Data Cleaning</span><span style='font-size:20px;color:#FFA500'>--</span>

In [ ]:
dataset['Country'].replace({'Mainland China':'China'},inplace=True)
last_updated['Country'].replace({'Mainland China':'China'},inplace=True)

In [ ]:
#removing the zero confirmed case rows
zeroConfirmed = dataset[dataset['Confirmed'] == 0]
dataset = dataset[dataset['Confirmed'] != 0]

In [ ]:
dataset.head()

In [ ]:
dataset[dataset['Date'] != dataset['Last Update']]['Country'].value_counts()

<span> These many province/State were not updated yet with last date.</span>

In [ ]:
dataset['Last Update'].max()

In [ ]:
# missing values
dataset.isnull().sum()

In [ ]:
confirmedCase = int(last_updated['Confirmed'].sum())
deathCase = int(last_updated['Deaths'].sum())
recoveredCase = int(last_updated['Recovered'].sum())
print("No of Confirmed cases globally {}".format(confirmedCase))
print("No of Recovered case globally {}".format(recoveredCase))
print("No of Death case globally {}".format(deathCase))

In [ ]:
others = dataset[dataset['Country']=='Others']
dataset = dataset[dataset['Country']!='Others']
last_updated =last_updated[last_updated['Country']!='Others']

We have <span style='color:red'> removed Others from Country Column</span>. We'll analyse it later

<span style='font-size:20px;color:#FFA500'>--</span><span style='font-size:20px'>Exploratory Data Analysis</span><span style='font-size:20px;color:#FFA500'>--</span>

In [ ]:
plt.figure(figsize=(15,6))
plt.title('Number of Province/State were affected in Each Country')
plt.xticks(rotation=90)
prv_lst = dataset.groupby(['Country'])['Province/State'].nunique().sort_values(ascending=False)
prv_lst.plot(kind='bar')
plt.tight_layout()

In [ ]:
prv_lst.tail()

Countries which shows Province/State Count zero which was because of Province/State were not updated for it. 

Hong Kong is not an independent country and is part of China.  However it is also not a province.<br>
Anyway <span style='background:red;color:white'>China has 33 Provinces excluding Hong Kong and coronavirus spreaded into all the provinces of China</span>. Even
Hong Kong also Affected by deadly dangerous virus.

In [ ]:
top5 =  last_updated.groupby(['Country']).sum().nlargest(5,['Confirmed'])[displayed_cols]
top5
print("Top 5 Countries were affected most")
print(top5)

<span style='color:blue'> Observation : </span>

Though US,Canada and Australia has higher entries after China but they are not in the top 5 list


In [ ]:
plt.figure(figsize=(12,6))
plt.xticks(rotation=90)
plt.title("Top most 5 countries were affected by Coronavirus")
sns.barplot(x=top5.index,y='Confirmed',data=top5)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
plt.title('Countries which has Confirmed cases')
plt.xticks(rotation=90)
sns.barplot(x='Country',y='Confirmed',data=last_updated)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(15,6))
plt.title('Province/State which reported more than 1000 Confirmed case')
plt.xticks(rotation=90)
prvinc = last_updated
prvincConfirmed = prvinc[prvinc['Confirmed']>100]
sns.barplot(data=prvincConfirmed, x='Province/State', y='Confirmed')

In [ ]:
prvincConfirmed['Country'].value_counts()

In [ ]:
plt.figure(figsize=(15,6))
plt.title('Province/State which reported Confirmed case between 100 to 1000')
plt.xticks(rotation=90)
prvinc = last_updated
prvincConfirmed = prvinc[(prvinc['Confirmed']>100)&(prvinc['Confirmed']<1000)]
sns.barplot(data=prvincConfirmed, x='Province/State', y='Confirmed')

In [ ]:
prvincConfirmed['Country'].value_counts()

In [ ]:
dataset.groupby(['Country']).nunique()

In [ ]:
plt.figure(figsize=(15,6))
plt.xticks(rotation=90)
plt.title('Province/State has reported Deaths case')
sns.barplot(data=last_updated, x='Province/State', y='Deaths')

Wikipedia - In December 2019, a pneumonia(Coronavirus) outbreak was reported in Wuhan, China.  <b><span style='font-size:16px;color:red'> Wuhan is the capital of Hubei province.</span></b>

In [ ]:
last_updated.groupby(['Country']).sum().nlargest(5,['Deaths'])['Deaths']

We'll analyse how other countries affected after China and Others(Cruis ship).

In [ ]:
last_updated[last_updated['Country']!='China'].groupby(['Country']).sum()[displayed_cols].nlargest(5,['Confirmed'])

In [ ]:
cruiseShip = others

In [ ]:
cruiseShip['Confirmed'].plot()

#### <span style='background:yellow'>confirmed case counts are increasing day by day in cruise ship</span>

Princess Cruises confirms there are 2,666 guests and 1,045 crew currently onboard covering a range of nationalities
Totally 3711 members are there in ship
 <span style='color:green'>Out of <b>3711 </b></span>, <span style='color:red'><b>454</b></span> were confirmed

In [ ]:
print("{}% of people were affected in Cruise ship".format(round((cruiseShip.iloc[-1].Confirmed/3711 ) * 100,2)))

Below Content taken from this article https://www.quora.com/What-is-the-Coronavirus-What-are-its-symptoms-How-do-you-prevent-the-Coronavirus answered by puneet aggarwal.

To prevent Coronavirus from spreading follow the below

1. Avoid using towels/soaps/ clothes used by any other person.

2. Keep cleaning your hands with a sanitizer especially after travelling in public transport like trains/ buses/ the metro etc.
3. <span style='background:yellow'>Keep yourself away from crowded areas.<span> 
4. <span style='background:yellow'>Always insist on wearing a mask especially use them for children.</span>
5. Drink only hot water

6. Keep your home, kitchen and bathroom dry. Viruses thrive in musty environments.

7. Have only fresh, hot, home cooked food.

<span style='font-size:25px;color:MediumSeaGreen'>Follow the highlighted steps, only if Coronavirus spreading in your Area and Other steps would be helpful to have Healthy Life.</span>